#LIBRARY

In [ ]:
%%capture
!pip install tensorflow

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split


#LOAD DATA

In [ ]:
from google.colab import drive
!gdown "1rD-_mHaqds2plYf3R20xe42NKKAEku49"

Downloading...
From: https://drive.google.com/uc?id=1rD-_mHaqds2plYf3R20xe42NKKAEku49
To: /content/databarulagi.csv
100% 48.8M/48.8M [00:01<00:00, 43.1MB/s]


In [ ]:
# Load Data
df = pd.read_csv("databarulagi.csv")
df.head()

,Unnamed: 0,index,full_text,sentimen,keyword,Cleaning,Casefolding,Alayfix,Tokenized,Change Word,Removed Stopwords,Untokenized,Stemmed
0,2906,3088,6342E+18;Surya Paloh Tak Masalah Jika Anies Ba...,1,anies,e surya paloh tak masalah jika anies baswedan ...,e surya paloh tak masalah jika anies baswedan ...,e surya paloh masalah anies baswedan pilih ahy...,"['e', 'surya', 'paloh', 'masalah', 'anies', 'b...","['e', 'surya', 'paloh', 'masalah', 'Anies', 'B...","['paloh', 'Anies', 'Baswedan', 'pilih', 'agus'...",paloh Anies Baswedan pilih agus harimurti yudh...,paloh anies baswedan pilih agus harimurti yudh...
1,8722,9612,"1708146700450440000 """"@lhayesno @ganjarpranowo...",1,anies,boleh tdk sy piniam narasi,boleh tdk sy piniam narasi,tdk sy piniam narasi,"['tdk', 'sy', 'piniam', 'narasi']","['tidak', 'saya', 'piniam', 'narasi']","['piniam', 'narasi']",piniam narasi,piniam narasi
2,13311,14840,@BanyuKirik @Yuliadi_Mr @ganjarpranowo @KAI121...,1,ganjar,jadi buzzer anies aja drun,jadi buzzer anies aja drun,jadi buzzer anies aja drun,"['jadi', 'buzzer', 'anies', 'aja', 'drun']","['jadi', 'buzzer', 'Anies', 'saja', 'kadrun']","['buzzer', 'Anies', 'kadrun']",buzzer Anies kadrun,buzzer anies kadrun
3,28000,33979,"Bansos itu disalurkan kepada masyarakat, agar ...",1,bansos,bansos itu salur kepada masyarakat agar bisa g...,bansos itu salur kepada masyarakat agar bisa g...,bansos salur masyarakat guna untuk penuh butuh...,"['bansos', 'salur', 'masyarakat', 'guna', 'unt...","['bantuan', 'sosial', 'salur', 'masyarakat', '...","['bantuan', 'sosial', 'salur', 'masyarakat', '...",bantuan sosial salur masyarakat berguna penuh ...,bantu sosial salur masyarakat guna penuh butuh...
4,42135,48916,"1597603508618160000 """"Survei Terbaru Indopol",1,anies,survei baru indopol,survei baru indopol,survei baru indopol,"['survei', 'baru', 'indopol']","['survei', 'baru', 'indopol']",['survei'],survei,survei


In [ ]:
df = df[df['sentimen'].isin([0, 2])]

In [ ]:
df_copy = df[['Stemmed','sentimen']]
df_copy.head()

,Stemmed,sentimen
18616,viral generasi bohong,0
18617,rusak nama anies baswedan jelek jelek nama anies,0
18618,bengkak,0
18619,kafir halang allah,0
18620,wajib presiden rakyat indonesia dukung preside...,0


In [ ]:
df['sentimen'].value_counts()

2    21866
0    15525
Name: sentimen, dtype: int64

#FEATURE EXTRACTION TFIDF

In [ ]:
X = df_copy["Stemmed"].astype(str)
Y = pd.get_dummies(df_copy['sentimen']).values

In [ ]:
le = LabelEncoder()

Y = le.fit_transform(df_copy['sentimen'])
Y = np.asarray(tf.keras.utils.to_categorical(Y, num_classes=2))

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.1, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(33651,) (33651, 2)
(3740,) (3740, 2)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=7, ngram_range=(1,2), max_features = 5000, norm='l1')
#vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit(X_train)
x_train_tfidf = vectorizer.fit_transform(X_train).todense()
x_test_tfidf = vectorizer.transform(X_test).todense()

# Matrix to Array
x_train = np.squeeze(np.asarray(x_train_tfidf))
x_train = x_train.reshape(-1, 1, x_train_tfidf.shape[1])

# Matrix to Array
x_test = np.squeeze(np.asarray(x_test_tfidf))
x_test = x_test.reshape(-1, 1, x_test_tfidf.shape[1])

In [ ]:
len(vectorizer.get_feature_names_out())

5000

In [ ]:
# Matrixx to array
x_train_22 = np.squeeze(np.asarray(x_train_tfidf))
x_train_22 = x_train_22.reshape(-1, 1, x_train_tfidf.shape[1])

# Matrixx to array
x_test_22 = np.squeeze(np.asarray(x_test_tfidf))
x_test_22 = x_test_22.reshape(-1, 1, x_test_tfidf.shape[1])

#GRU

In [ ]:
from keras.layers import GRU
from keras.models import Sequential
from keras.layers import Dense,Bidirectional,Embedding,Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

# model
# del model
def GRU_model(x_train_c, y_train_c, x_test_c, y_test_c):
    input_shape = x_train_c.shape

    model2 = Sequential()
    model2.add(GRU(256, return_sequences=True, activation='relu'))
    model2.add(Dropout(0.5))
    model2.add(GRU(128, return_sequences=True, activation='relu'))
    model2.add(Dropout(0.5))
    model2.add(GRU(64, return_sequences=False, activation='relu'))
    model2.add(Dropout(0.5))
    #model2.add(Dense(64,activation='tanh'))
    #model2.add(Dense(64,activation='tanh'))
    #model2.add(Dense(10,activation='tanh'))
    model2.add(Dense(2,activation='softmax'))

    model2.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
    model2.build(input_shape)

    batch_size = 128

    es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 5)
    mc = ModelCheckpoint('./model.h5', monitor = 'val_accuracy', mode = 'max', verbose = 1, save_best_only = True)

    history = model2.fit(x_train_c, y_train_c, epochs = 10, batch_size=batch_size,
                    validation_data=(x_test_c, y_test_c), verbose = 1,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=5, min_delta=0.0001)])

    y_pred = model2.predict(x_test_c)
    y_pred_cat = y_pred.argmax(axis=1)
    y_test_cat = np.argmax(y_test_c, axis=1)

    acc = accuracy_score(y_test_cat, y_pred_cat)
    f1 = f1_score(y_test_cat, y_pred_cat,  average="macro")
    recall = recall_score(y_test_cat, y_pred_cat, average="macro")
    prec = precision_score(y_test_cat, y_pred_cat, average="macro")

    print(accuracy_score(y_test_cat, y_pred_cat))
    print(classification_report(y_test_cat, y_pred_cat))
    print("Precision: ",precision_score(y_test_cat, y_pred_cat, average="macro"))
    print("Recall: ",recall_score(y_test_cat, y_pred_cat, average="macro"))
    print("F1: ",f1_score(y_test_cat, y_pred_cat,  average="macro"))
    print("Accuracy: ",accuracy_score(y_test_cat, y_pred_cat))

    return [acc, f1, recall, prec]


In [ ]:
GRU_model(x_train_22, Y_train, x_test_22, Y_test)

Epoch 1/10
263/263 [==============================] - 11s 19ms/step - loss: 0.6768 - accuracy: 0.5846 - val_loss: 0.6571 - val_accuracy: 0.5824
Epoch 2/10
263/263 [==============================] - 5s 18ms/step - loss: 0.5255 - accuracy: 0.7392 - val_loss: 0.4172 - val_accuracy: 0.8243
Epoch 3/10
263/263 [==============================] - 4s 14ms/step - loss: 0.4202 - accuracy: 0.8189 - val_loss: 0.3979 - val_accuracy: 0.8291
Epoch 4/10
263/263 [==============================] - 9s 33ms/step - loss: 0.3888 - accuracy: 0.8387 - val_loss: 0.3965 - val_accuracy: 0.8299
Epoch 5/10
263/263 [==============================] - 5s 18ms/step - loss: 0.3724 - accuracy: 0.8499 - val_loss: 0.3971 - val_accuracy: 0.8241
Epoch 6/10
263/263 [==============================] - 5s 17ms/step - loss: 0.3580 - accuracy: 0.8576 - val_loss: 0.3954 - val_accuracy: 0.8283
Epoch 7/10
263/263 [==============================] - 4s 14ms/step - loss: 0.3444 - accuracy: 0.8646 - val_loss: 0.3947 - val_accuracy: 0.832

[0.8350267379679145,
 0.8305029594295812,
 0.8306528796285518,
 0.8303562224915575]

In [ ]:
%%time
hasil = []

# looping dari 1 - 5 (5 kali run)
for i in range(1,6):
    hasil.append([str(i)] + GRU_model(x_train_22, Y_train, x_test_22, Y_test))

Epoch 1/10
263/263 [==============================] - 8s 16ms/step - loss: 0.6787 - accuracy: 0.5848 - val_loss: 0.6737 - val_accuracy: 0.5824
Epoch 2/10
263/263 [==============================] - 3s 13ms/step - loss: 0.5773 - accuracy: 0.6864 - val_loss: 0.4377 - val_accuracy: 0.8136
Epoch 3/10
263/263 [==============================] - 5s 17ms/step - loss: 0.4358 - accuracy: 0.8076 - val_loss: 0.4087 - val_accuracy: 0.8211
Epoch 4/10
263/263 [==============================] - 4s 14ms/step - loss: 0.3952 - accuracy: 0.8344 - val_loss: 0.3971 - val_accuracy: 0.8275
Epoch 5/10
263/263 [==============================] - 3s 12ms/step - loss: 0.3748 - accuracy: 0.8478 - val_loss: 0.3945 - val_accuracy: 0.8286
Epoch 6/10
263/263 [==============================] - 5s 18ms/step - loss: 0.3590 - accuracy: 0.8558 - val_loss: 0.3930 - val_accuracy: 0.8345
Epoch 7/10
263/263 [==============================] - 4s 15ms/step - loss: 0.3481 - accuracy: 0.8631 - val_loss: 0.3945 - val_accuracy: 0.8324

Precision:  0.8223432826920607
Recall:  0.830494445091116
F1:  0.823630789420162
Accuracy:  0.825668449197861
Epoch 1/10
263/263 [==============================] - 9s 19ms/step - loss: 0.6778 - accuracy: 0.5846 - val_loss: 0.6654 - val_accuracy: 0.5824
Epoch 2/10
263/263 [==============================] - 3s 13ms/step - loss: 0.5452 - accuracy: 0.7161 - val_loss: 0.4386 - val_accuracy: 0.8040
Epoch 3/10
263/263 [==============================] - 6s 21ms/step - loss: 0.4223 - accuracy: 0.8151 - val_loss: 0.4046 - val_accuracy: 0.8251
Epoch 4/10
263/263 [==============================] - 3s 13ms/step - loss: 0.3923 - accuracy: 0.8371 - val_loss: 0.3964 - val_accuracy: 0.8278
Epoch 5/10
263/263 [==============================] - 3s 13ms/step - loss: 0.3711 - accuracy: 0.8489 - val_loss: 0.3946 - val_accuracy: 0.8281
Epoch 6/10
263/263 [==============================] - 3s 13ms/step - loss: 0.3586 - accuracy: 0.8566 - val_loss: 0.3974 - val_accuracy: 0.8291
Epoch 7/10
263/263 [============

0.8304812834224599
              precision    recall  f1-score   support

           0       0.79      0.82      0.80      1562
           1       0.86      0.84      0.85      2178

    accuracy                           0.83      3740
   macro avg       0.83      0.83      0.83      3740
weighted avg       0.83      0.83      0.83      3740

Precision:  0.8252768025924926
Recall:  0.8285608970628178
F1:  0.8266782567304383
Accuracy:  0.8304812834224599
Epoch 1/10
263/263 [==============================] - 9s 19ms/step - loss: 0.6773 - accuracy: 0.5841 - val_loss: 0.6606 - val_accuracy: 0.5824
Epoch 2/10
263/263 [==============================] - 3s 13ms/step - loss: 0.5227 - accuracy: 0.7390 - val_loss: 0.4332 - val_accuracy: 0.8032
Epoch 3/10
263/263 [==============================] - 3s 13ms/step - loss: 0.4184 - accuracy: 0.8187 - val_loss: 0.3983 - val_accuracy: 0.8286
Epoch 4/10
263/263 [==============================] - 5s 21ms/step - loss: 0.3895 - accuracy: 0.8374 - val_loss:

0.8254010695187166
              precision    recall  f1-score   support

           0       0.76      0.86      0.80      1562
           1       0.89      0.80      0.84      2178

    accuracy                           0.83      3740
   macro avg       0.82      0.83      0.82      3740
weighted avg       0.83      0.83      0.83      3740

Precision:  0.8222114549843952
Recall:  0.8304459447225132
F1:  0.8234113095229058
Accuracy:  0.8254010695187166
Epoch 1/10
263/263 [==============================] - 9s 17ms/step - loss: 0.6753 - accuracy: 0.5846 - val_loss: 0.6454 - val_accuracy: 0.5824
Epoch 2/10
263/263 [==============================] - 4s 14ms/step - loss: 0.5160 - accuracy: 0.7472 - val_loss: 0.4201 - val_accuracy: 0.8176
Epoch 3/10
263/263 [==============================] - 5s 18ms/step - loss: 0.4172 - accuracy: 0.8218 - val_loss: 0.3982 - val_accuracy: 0.8294
Epoch 4/10
263/263 [==============================] - 4s 17ms/step - loss: 0.3883 - accuracy: 0.8397 - val_loss:

F1:  0.8266218863143338
Accuracy:  0.8304812834224599
Epoch 1/10
263/263 [==============================] - 10s 21ms/step - loss: 0.6766 - accuracy: 0.5847 - val_loss: 0.6538 - val_accuracy: 0.5824
Epoch 2/10
263/263 [==============================] - 4s 16ms/step - loss: 0.5181 - accuracy: 0.7425 - val_loss: 0.4178 - val_accuracy: 0.8198
Epoch 3/10
263/263 [==============================] - 4s 16ms/step - loss: 0.4173 - accuracy: 0.8196 - val_loss: 0.3996 - val_accuracy: 0.8238
Epoch 4/10
263/263 [==============================] - 6s 21ms/step - loss: 0.3862 - accuracy: 0.8403 - val_loss: 0.3938 - val_accuracy: 0.8267
Epoch 5/10
263/263 [==============================] - 4s 15ms/step - loss: 0.3696 - accuracy: 0.8503 - val_loss: 0.3959 - val_accuracy: 0.8259
Epoch 6/10
263/263 [==============================] - 4s 14ms/step - loss: 0.3553 - accuracy: 0.8572 - val_loss: 0.3959 - val_accuracy: 0.8235
Epoch 7/10
263/263 [==============================] - 6s 23ms/step - loss: 0.3425 - acc

In [ ]:
df = pd.DataFrame(hasil, columns =['itr', 'acc', 'f1', 'recall', 'precision'])

# save to excel
df.to_excel('tfidf.xlsx')
df

,itr,acc,f1,recall,precision
0,1,0.825668,0.823631,0.830494,0.822343
1,2,0.830481,0.826678,0.828561,0.825277
2,3,0.825401,0.823411,0.830446,0.822211
3,4,0.830481,0.826622,0.828380,0.825285
4,5,0.833155,0.829014,0.830042,0.828133


In [ ]:
print("Accuracy : %.2f" % (df['acc'].mean()*100))
print("F1-Score : %.2f" % (df['f1'].mean()*100))

Accuracy : 82.90
F1-Score : 82.59
